*WormsPy Post Processing Pipeline*

This Jupyter notebook contains code to extract data from WormsPy outputs into CSV data in the following code blocks:
1. Formatting step that makes it so a DLC node can be superimposed onto stage position for more exact tracking (of the nose tip for example)
2. Master imports and directories for the WormsPy recording you are working on
3. Converts folder of tiffs into one tif stack that is the input for the provided segmentation code (separate python file)
4. Auto-annotate from curvature and angle data
5. Nose-tip translation from DLC file onto stage position for absolute position.
6. Combine datafiles.

As inputs it requires the folder of .tiff files, the CSV of stage position, and the brightfield .avi produced by a WormsPy recording.

As an output it produces a .csv file with mean and sum of fluorescence changes within the ROI, the position of the nose of the animal, and worm curvatures to auto-annotate reversals.

Format all DLC CSVs correctly:

In [ ]:
import os
import pandas as pd # type: ignore
# Define the folder containing the CSV files
folder_path = 'DLC_FOLDER'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        if filename.startswith('._'):
            continue
        file_path = os.path.join(folder_path, filename)
        print(f'Processing file: {file_path}')
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Keep only the first three columns
        df = df.iloc[:, :3]

        # drop first two rows
        df = df.drop([0, 1])
        
        # Rename the columns
        df.columns = ['index', 'x', 'y']

        # Convert 'x' and 'y' columns to numeric types
        df['x'] = pd.to_numeric(df['x'], errors='coerce')
        df['y'] = pd.to_numeric(df['y'], errors='coerce')
        
        # Calculate the differences for X and Y columns
        df['X_diff'] = df['x'].diff().abs()
        df['Y_diff'] = df['y'].diff().abs()

        # Replace values where the difference is above 30 pixels
        df.loc[df['X_diff'] > 30, 'x'] = df['x'].shift()
        df.loc[df['Y_diff'] > 30, 'y'] = df['y'].shift()

        # smooth the series
        df['x'] = df['x'].rolling(window=3, min_periods=1).mean()
        df['y'] = df['y'].rolling(window=3, min_periods=1).mean()

        # round the values to 0 decimal places
        df['x'] = df['x'].round(0)
        df['y'] = df['y'].round(0)
        
        # Save the modified DataFrame back to a CSV file
        df.to_csv(file_path, index=False)

Imports for all code blocks

In [27]:
import os
import tifffile
import numpy as np
import re
import cv2
import csv
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

directory = "DIRECTORY" # directory of the WormsPy output folder
name = "NAME" # desired name
stagepos_csv = os.path.join(directory, name + '_stagepos.csv') # input file
annotatedcsv = os.path.join(directory, name + '_annotated.csv') # output file
segmented_csv_path = os.path.join(directory, name + '_segmented.csv') #input file


# Read the DLC CSV file
DLC_file = os.path.join(directory, name + 'DLC.csv') # input file
nosepos_csv = os.path.join(directory, name + '_nosecoords.csv') #output file

Tiffstacker: This script converts a folder of images into a single .tiff stack that can by dynamically segmented. Run Segmentation scipt on this tiff stack.

In [ ]:
def numerical_sort(value):
    numbers = re.findall(r'\d+', value)
    return int(numbers[0]) if numbers else 0

def tiff_stacker(dir_path, output_path):
    # Get a list of all TIFF files in the directory
    tiff_files = [file for file in os.listdir(dir_path) if file.endswith('.tiff')]

    # Sort the TIFF files numerically
    tiff_files.sort(key=numerical_sort)

    # Read the first image to determine its shape and data type
    first_image = tifffile.imread(os.path.join(dir_path, tiff_files[0]))

    # Initialize an empty stack with appropriate dimensions and data type
    stack = np.zeros((len(tiff_files), first_image.shape[0], first_image.shape[1]), dtype=first_image.dtype)

    # Read each TIFF file and add it to the stack
    for i, tiff_file in enumerate(tiff_files):
        image = tifffile.imread(os.path.join(dir_path, tiff_file))
        stack[i] = image

    # Save the stack as a multi-image TIFF file
    tifffile.imwrite(output_path, stack)

    print(f"Multi-image TIFF stack saved at {output_path}")

for foldername in os.listdir(directory):
    input_path = os.path.join(directory, foldername)
    if os.path.isdir(input_path):  # Ensure input_path is a directory
        output_path = os.path.join(directory, name + ".tiff") #output name
        tiff_stacker(input_path, output_path)
    else:
        print(f"{input_path} is not a directory")

Auto-Annotate: Takes the CSV of stage position recordings and calculates the angle and curvature for each frame to find instances of likely reversals. Recommended you manually check the annotations and filter out false positives. 

In [ ]:
def get_vertex_angles(x, y):
    n = len(x)
    if n < 3:
        raise ValueError('At least three points are required to calculate angles.')
    
    angles = np.zeros(n-2)
    
    for i in range(1, n-1):
        BA_x = x[i-1] - x[i]
        BA_y = y[i-1] - y[i]
        BC_x = x[i+1] - x[i]
        BC_y = y[i+1] - y[i]
        
        dot_product = BA_x * BC_x + BA_y * BC_y
        magnitude_BA = np.sqrt(BA_x**2 + BA_y**2)
        magnitude_BC = np.sqrt(BC_x**2 + BC_y**2)
        
        cos_theta = dot_product / (magnitude_BA * magnitude_BC)
        angle_radians = np.arccos(cos_theta)
        angles[i-1] = np.degrees(angle_radians)
    
    return angles

def calculate_three_point_curvatures(x, y):
    n = len(x)
    curvatures = np.zeros(n-2)
    
    for i in range(1, n-1):
        x1, y1 = x[i-1], y[i-1]
        x2, y2 = x[i], y[i]
        x3, y3 = x[i+1], y[i+1]
        
        num = abs((x2 - x1) * (y3 - y1) - (y2 - y1) * (x3 - x1))
        den = np.sqrt(((x2 - x1)**2 + (y2 - y1)**2) * ((x3 - x1)**2 + (y3 - y1)**2) * ((x3 - x2)**2 + (y3 - y2)**2))
        
        if den != 0:
            curvatures[i-1] = 2 * num / den
        else:
            curvatures[i-1] = 0
    
    return curvatures

# edit angle_threshold and curvature_threshold as needed
def auto_annotate(ax, x, y, angle_threshold=10, curvature_threshold=0.05):
    angles = get_vertex_angles(x, y)
    curvatures = calculate_three_point_curvatures(x, y)
    
    n = len(x)
    
    for i in range(1, n-1):
        if angles[i-1] > angle_threshold:
            ax.plot(x[i], y[i], 'bo', markersize=5)
        
        if curvatures[i-1] > curvature_threshold:
            ax.plot(x[i], y[i], 'ro', markersize=5)

if __name__ == '__main__':
    print(stagepos_csv)
    data = pd.read_csv(stagepos_csv)
    x = data['X_motor'].values
    y = data['Y_motor'].values

    fig, ax = plt.subplots()
    auto_annotate(ax, x, y)
    plt.show()

    data = data.drop([0, 1])
    
    curvatures = calculate_three_point_curvatures(x, y)
    data['Curvatures'] = np.concatenate((np.array([0]), curvatures[:-2], np.array([0])))
    
    angles = get_vertex_angles(x, y)
    data['Angles'] = np.concatenate((np.array([0]), angles[:-2], np.array([0])))
    angle_threshold=30
    curvature_threshold=0.03
    data['Reversals'] = ((data['Curvatures'] > curvature_threshold) & (data['Angles'] < angle_threshold)).astype(int)

    data.to_csv(annotatedcsv, index=False)

For finer tuned behavioural annotation we recommend using DeepLabCut that can be trained for your specifications.

Nose Tip Translation: This code takes the pixel coordinates of a point of interest from DLC (like the nose tip) and adds it onto the stage position in uM. DLC CSV will need to be formatted to drop the unnecessary rows and drop all columns besides the X and Y for the desired point.

In [ ]:
# Define the midpoint of the video (resolution / 2) CHANGE IF NEEDED
midpoint_x = 960
midpoint_y = 600
pxtouM = 1.54
 
# Open the DLC file and read the X and Y columns, subtracting midpoint_x and midpoint_y
with open(DLC_file, 'r') as dlc_csv:
    dlc_reader = csv.DictReader(dlc_csv)
    dlc_data = [((float(row['x']) - midpoint_x)*pxtouM, ((float(row['y']) - midpoint_y)*pxtouM)) for row in dlc_reader]    

# Open the stage file and read the data
with open(annotatedcsv, 'r') as stage_csv:
    stage_reader = csv.DictReader(stage_csv)
    stage_data = [row for row in stage_reader]
    stage_fieldnames = stage_reader.fieldnames

# Add new columns for X_nose and Y_nose
stage_fieldnames.extend(['X_nose', 'Y_nose'])

# Combine the data and write to a new CSV file
with open(nosepos_csv, 'w', newline='') as output_csv:
    writer = csv.DictWriter(output_csv, fieldnames=stage_fieldnames)
    writer.writeheader()
    for i, row in enumerate(stage_data):
        row['X_nose'] = float(row['X_motor']) + dlc_data[i][0]
        row['Y_nose'] = float(row['Y_motor']) - dlc_data[i][1]
        writer.writerow(row)

Combine the position data with the segmentation data:

In [22]:
# Read the nose position CSV
position_data = pd.read_csv(nosepos_csv)

# Read the segmentation CSV
segmentation_data = pd.read_csv(segmented_csv_path)

# Join the two data tables based on row number
combined_data = pd.concat([segmentation_data, position_data], axis=1)

# Write the combined data to a new CSV
combined_csv_path = os.path.join(directory, name + "_combined.csv")
combined_data.to_csv(combined_csv_path, index=False)